# Applied Neural Networks - Exercises 

**NOTICE:**
1. You are allowed to work in groups of up to three people but **have to document** your group's\
 members in the top cell of your notebook.
2. **Comment your code**, explain what you do (refer to the slides). It will help you understand the topics\
 and help me understand your thinking progress. Quality of comments will be graded. 
3. **Discuss** and analyze your results, **write-down your learnings**. These exercises are no programming\
 exercises it is about learning and getting a touch for these methods. Such questions might be asked in the\
 final exams. 
 4. Feel free to **experiment** with these methods. Change parameters think about improvements, write down\
 what you learned. This is not only about collecting points for the final grade, it is about understanding\
  the methods. 

### Exercise 1 - Data Normalization and Standardization


**Summary:** In this exercise you will implement the min-max normalization and standardization and compare it to\
sklearn's implementation. It is important to remember, that we always normalize or standardize for all samples\
 over a single feature dimension.


**Provided Code:** In the cell below I have provided you with a sample code to initialize some dummy data.\
The parameter ```n_samples``` defines the number of samples we have in the training set (the number of $x_i$)\
while ```n_features``` defines the number of dimensions of each sample feature vector. 


**Your Tasks in this exercise:**
1. Implement the MinMax Normalization and Standardization. 
2. Use the ```MinMaxScaler``` and ```StandardScaler``` from sklearn to verify your results. 


In [170]:
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler

x,y = make_regression(n_samples=10, n_features=5)

### Exercise 2 - Softmax

**Summary:** In this exercise you will implement the softmax activation using the naive and numerically\
more stable log-sum variation. 


**Provided Code:** In the cell below there is some sample code that generates sample inputs. 


**Your Tasks in this exercise:**
1. Implement the softmax function using the naive approach. 
2. Implement the softmax function using the log-sum trick. 
3. Compare your two implementations for numerical stability\
(experiment with different values of std) and verify
your results using ```tf.nn.softmax```



In [8]:
import numpy as np 
import tensorflow as tf

mu = 0
std = 10
xi = mu + std * np.random.randn(10)

### Exercise 3 - Chess Endgames

**Summary:** In this exercise your task is to predict the optimal depth-of-win for white in   
chess-endgames. In particular, we will focus on **king-rook** vs. **king** endgames. The   
possible outcomes are either a **draw** or a **number of moves** for white to win (0 to 16). 


**Provided Code:** The code below loads the original (*unprepared*) raw dataset.   
You will have to prepare it accordingly to be used with neural nets. 

The structure of each row in the dataset is:
1. White King column (a-h)
2. White King row (1-8)
3. White Rook column (a-h)
4. White Rook row (1-8)
5. Black King column (a-h)
6. Black King row (1-8)
7. Optimal depth-of-win for White in 0 to 16 moves or a draw


**Your Tasks in this exercise:**
1. Train a neural net to predict the depth-of-win (or draw) given a board position
    * You will have to prepare your data accordingly to make it compatible   
    with neural nets. Think about input and output encodings, normalization or standardization. 
    * Decide how you will model this problem as either regression or classification task. 
    * Build a fully connected neural net with appropriate configuration and loss and train it. 
    * Use appropriate cross-validation for training and validation (it is enough to use two datasets)
2. Explain in writing:
    * How and why did you prepared the data?
    * How did you model the problem task?
    * What is your neural network architecture/configuration/loss?
    * Plot your loss while training.
    * Interpret and explain your results. 
    



In [97]:
!wget https://github.com/shegenbart/Jupyter-Exercises/raw/main/data/condition_monitoring.pickle -P ../data
import pickle
with open('../data/chess_endgames.pickle', 'rb') as fd:
    chess_endgames = pickle.load(fd)


Der Befehl "wget" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


### Solution 3 - Chess Endgames

In [63]:
col_lookup  = {'a': 0, 'b' : 1, 'c': 2, 'd' : 3, 'e': 4, 'f': 5, 'g':6, 'h':7}
class_lookup = {'draw' : 0, 'zero' : 1, 'one': 2, 'two' : 3, 'three' : 4, 'four': 5, 
                'five': 6, 'six' : 7, 'seven' : 8, 'eight' : 9, 'nine' : 10,
                'ten' : 11, 'eleven' : 12, 'twelve' : 13, 'thirteen' : 14, 
                'fourteen' : 15, 'fifteen' : 16, 'sixteen' : 17}


y = rows[:,-1]
x = rows[:, 0:-1]

for row in x:
    row[0] = col_lookup[row[0]]
    row[2] = col_lookup[row[2]]
    row[4] = col_lookup[row[4]]


for i in range(len(y)):
    y[i] = class_lookup[y[i]]
    

x = x.astype('uint32')
y = y.astype('uint32')
y = tf.keras.utils.to_categorical(y)
    

    

In [67]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

In [74]:
# normalize input data
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from tensorflow.keras.regularizers import L2
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.layers import LeakyReLU

model = tf.keras.Sequential()
model.add(Input(shape=(6)))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(18,  activation='softmax'))


model.summary()
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=25, min_lr=0.00001)

]

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 256)               1792      
                                                                 
 dense_29 (Dense)            (None, 256)               65792     
                                                                 
 dense_30 (Dense)            (None, 256)               65792     
                                                                 
 dense_31 (Dense)            (None, 256)               65792     
                                                                 
 dense_32 (Dense)            (None, 256)               65792     
                                                                 
 dense_33 (Dense)            (None, 18)                4626      
                                                                 
Total params: 269,586
Trainable params: 269,586
Non-tr

In [110]:
history = model.fit(x_train,y_train, validation_data=(x_test, y_test), epochs=100, batch_size=32)

Epoch 1/100
 10/614 [..............................] - ETA: 3s - loss: 0.1436 - accuracy: 0.9469 

614/614 [==============================] - 2s 3ms/step - loss: 0.0952 - accuracy: 0.9692 - val_loss: 0.5580 - val_accuracy: 0.8692
Epoch 2/100
614/614 [==============================] - 2s 3ms/step - loss: 0.0883 - accuracy: 0.9701 - val_loss: 0.5397 - val_accuracy: 0.8735
Epoch 3/100
473/614 [======================>.......] - ETA: 0s - loss: 0.0818 - accuracy: 0.9718

: 

In [1]:
import matplotlib.pyplot as plt 

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train-loss', 'val-loss'])

NameError: name 'history' is not defined

: 